# Mental Health in Tech

How must we shape the tech industry moving forward to support people from different backgrounds in regards to their mental health? Are there patterns among people who seek treatment and those who do not? How do attitudes toward mental health vary and does this variation have implications for seeking help?



Pathways of analysis:
- Predict whether someone has been treated for a mental illness based on key features found in preprocessing 
- Question: how do we want to structure the training & test set for our methods by each survey or use 2014 as a training set & 2016 as a test set?

Methods:

1. Preprocessing data
    - Clean columns (regex?) & create code book so info across tables to match 
    - feature engineering & identify important columns
    - group & visualize demographic make up of people in study
    - test/train split within same survey year
2.  Use decision trees to classify individuals as seeking treatment or not seeking treatment for mental health illnesses
    - use cross validation to refine model
    - accuracy, precision, Area Under the ROCurve (AUC)
    - visualizations
3. Use ensemble methods to merge several decision trees based on different features
    - bagging, random forests
    - visualizations
4. Use k-means clustering to identify groups of similar workers & check if similar workers (do/do not) engage in mental health treatment
     - use cross validation to refine model
     - silhouette score & elbow method
     - visualizations


In [ ]:
# Importing Libraries
!pip install --upgrade pip
!pip install seaborn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# The Data

To approach these questions, we are using mental health data from surveys run by Open Sourcing Mental Illness (OSMI) in 2014 & 2016. 

## Mental Health in Tech Survey (2016)

The dataset is found [here](https://www.kaggle.com/osmi/mental-health-in-tech-2016?select=mental-heath-in-tech-2016_20161114.csv). It has 1433 survey respondents from 2016. 839 of the respondents stated that they had received treatment  for a mental health issue, while 594 had not.

In [ ]:
df = pd.read_csv("mental-heath-in-tech-2016_20161114.csv")
pd.set_option('display.max_columns', None)
df.head()

,Are you self-employed?,How many employees does your company or organization have?,Is your employer primarily a tech company/organization?,Is your primary role within your company related to tech/IT?,Does your employer provide mental health benefits as part of healthcare coverage?,Do you know the options for mental health care available under your employer-provided coverage?,"Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?",Does your employer offer resources to learn more about mental health concerns and options for seeking help?,Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?,"If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:",Do you think that discussing a mental health disorder with your employer would have negative consequences?,Do you think that discussing a physical health issue with your employer would have negative consequences?,Would you feel comfortable discussing a mental health disorder with your coworkers?,Would you feel comfortable discussing a mental health disorder with your direct supervisor(s)?,Do you feel that your employer takes mental health as seriously as physical health?,Have you heard of or observed negative consequences for co-workers who have been open about mental health issues in your workplace?,Do you have medical coverage (private insurance or state-provided) which includes treatment of mental health issues?,Do you know local or online resources to seek help for a mental health disorder?,"If you have been diagnosed or treated for a mental health disorder, do you ever reveal this to clients or business contacts?","If you have revealed a mental health issue to a client or business contact, do you believe this has impacted you negatively?","If you have been diagnosed or treated for a mental health disorder, do you ever reveal this to coworkers or employees?","If you have revealed a mental health issue to a coworker or employee, do you believe this has impacted you negatively?",Do you believe your productivity is ever affected by a mental health issue?,"If yes, what percentage of your work time (time performing primary or secondary job functions) is affected by a mental health issue?",Do you have previous employers?,Have your previous employers provided mental health benefits?,Were you aware of the options for mental health care provided by your previous employers?,Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?,Did your previous employers provide resources to learn more about mental health issues and how to seek help?,Was your anonymity protected if you chose to take advantage of mental health or substance abuse treatment resources with previous employers?,Do you think that discussing a mental health disorder with previous employers would have negative consequences?,Do you think that discussing a physical health issue with previous employers would have negative consequences?,Would you have been willing to discuss a mental health issue with your previous co-workers?,Would you have been willing to discuss a mental health issue with your direct supervisor(s)?,Did you feel that your previous employers took mental health as seriously as physical health?,Did you hear of or observe negative consequences for co-workers with mental health issues in your previous workplaces?,Would you be willing to bring up a physical health issue with a potential employer in an interview?,Why or why not?,Would you bring up a mental health issue with a potential employer in an interview?,Why or why not?.1,Do you feel that being identified as a person with a mental health issue would hurt your career?,Do you think that team members/co-workers would view you more negatively if they knew you suffered from a menta

In [ ]:
df.columns

Index(['Are you self-employed?',
       'How many employees does your company or organization have?',
       'Is your employer primarily a tech company/organization?',
       'Is your primary role within your company related to tech/IT?',
       'Does your employer provide mental health benefits as part of healthcare coverage?',
       'Do you know the options for mental health care available under your employer-provided coverage?',
       'Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?',
       'Does your employer offer resources to learn more about mental health concerns and options for seeking help?',
       'Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?',
       'If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:',
       'Do you think that dis

In [ ]:
df.groupby('Have you ever sought treatment for a mental health issue from a mental health professional?').size()

# Pre-Processing

This large dataset includes a lot of missing information, integrated data, inconsistency regarding number of respondents, and data that indicate the same thing but aren't uniform. Thus, a lot of data cleaning must be done first in order to have presentable data to pass into our models.

For ease in working with the data later, columns will be renamed to shorter / more succint labels. 

In [ ]:
renamed = ["Self Employed", "Total Employees", "Tech/No Tech", "Tech Role",
           "MH Benefits", "Awareness of Company MH Care", "Formal MH Discussion",
           "MH Resources", "Anonymity", "Medical Leave", "MH Negative Impact",
           "PH Negative Impact", "Telling Coworkers", "Telling Supervisors",
           "MH PH Equal", "Observed Negative Impact", "Medical Coverage",
           "Awareness of Local Resources", "Reveal Clients (If Diagnosed)",
           "Negative Impact (If Revealed Clients)", "Reveal Employees (If Diagnosed)", 
           "Negative Impact (If Revealed Employees)", "Affected Productivity",
           "% Affected", "Previous Employers", "Prev MH Benefits", "Prev Awareness of Company MH Care",
           "Prev Formal MH Discussion", "Prev MH Resources", "Prev Anonymity",
           "Prev MH Negative Impact", "Prev PH Negative Impact", "Prev Telling Coworkers",
           "Prev Telling Supervisors", "Prev MH PH Equal", "Prev Observed Negative Impact",
           "PH Potential Employer", "PH Why/Not", "MH Potential Employer", "MH Why/Not",
           "MH Identification Career Harm", "Negative Impact Coworkers", "Sharing w/ Family/Friends",
           "Prev Observed Unsupportive Response", "Less Likely to Reveal", "MH Family History",
           "Past Mental Disorder", "Current Mental Disorder", "Condition", "Maybe Condition",
           "Professional Diagnosis", "Diagnosis", "Sought Treatment",
           "MH Interference Treatment", "MH Interference No Treatment", "Age",
           "Gender", "Country Live", "State Live", "Country", "State", "Position", "Remote"]
df.columns = renamed
df

,Self Employed,Total Employees,Tech/No Tech,Tech Role,MH Benefits,Awareness of Company MH Care,Formal MH Discussion,MH Resources,Anonymity,Medical Leave,MH Negative Impact,PH Negative Impact,Telling Coworkers,Telling Supervisors,MH PH Equal,Observed Negative Impact,Medical Coverage,Awareness of Local Resources,Reveal Clients (If Diagnosed),Negative Impact (If Revealed Clients),Reveal Employees (If Diagnosed),Negative Impact (If Revealed Employees),Affected Productivity,% Affected,Previous Employers,Prev MH Benefits,Prev Awareness of Company MH Care,Prev Formal MH Discussion,Prev MH Resources,Prev Anonymity,Prev MH Negative Impact,Prev PH Negative Impact,Prev Telling Coworkers,Prev Telling Supervisors,Prev MH PH Equal,Prev Observed Negative Impact,PH Potential Employer,PH Why/Not,MH Potential Employer,MH Why/Not,MH Identification Career Harm,Negative Impact Coworkers,Sharing w/ Family/Friends,Prev Observed Unsupportive Response,Less Likely to Reveal,MH Family History,Past Mental Disorder,Current Mental Disorder,Condition,Maybe Condition,Professional Diagnosis,Diagnosis,Sought Treatment,MH Interference Treatment,MH Interference No Treatment,Age,Gender,Country Live,State Live,Country,State,Position,Remote
0,0,26-100,1.0,NaN,Not eligible for coverage / N/A,NaN,No,No,I don't know,Very easy,No,No,Maybe,Yes,I don't know,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,"No, none did",N/A (not currently aware),I don't know,None did,I don't know,Some of them,None of them,Some of my previous employers,Some of my previous employers,I don't know,None of them,Maybe,NaN,Maybe,NaN,Maybe,"No, I don't think they would",Somewhat open,No,NaN,No,Yes,No,NaN,NaN,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",0,Not applicable to me,Not applicable to me,39,Male,United Kingdom,NaN,United Kingdom,NaN,Back-end Developer,Sometimes
1,0,6-25,1.0,NaN,No,Yes,Yes,Yes,Yes,Somewhat easy,No,No,Maybe,Yes,Yes,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,"Yes, they all did",I was aware of some,None did,Some did,"Yes, always",None of them,None of them,"No, at none of my previous employers",Some of my previous employers,Some did,None of them,Maybe,It would depend on the health issue. If there ...,No,While mental health has become a more prominen...,"No, I don't think it would","No, I don't think they would",Somewhat open,No,NaN,Yes,Yes,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",NaN,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",1,Rarely,Sometimes,29,male,United States of America,Illinois,United States of America,Illinois,Back-end Developer|Front-end Developer,Never
2,0,6-25,1.0,NaN,No,NaN,No,No,I don't know,Neither easy nor difficult,Maybe,No,Maybe,Maybe,I don't know,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,"No, none did",N/A (not currently aware),None did,Some did,I don't know,I don't know,Some of them,Some of my previous employers,I don't know,I don't know,Some of them,Yes,"They would provable need to know, to Judge if ...",Yes,"Stigma, mainly.",Maybe,Maybe,Somewhat open,Maybe/Not sure,Yes,No,Maybe,No,NaN,NaN,No,NaN,1,Not applicable to me,Not applicable to me,38,Male,United Kingdom,NaN,United Kingdom,NaN,Back-end Developer,Always
3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,"Yes, I know several","Sometimes, if it comes up",I'm not sure,"Sometimes, if it comes up",I'm not sure,Yes,1-25%,1,Some did,N/A (not currently aware),None did,None did,I don't know,Some of them,Some of them,Some of my previous employers,Some of my previous employers,I don't know,Some of them,Yes,"old back injury, doesn't cause me many issues ...",Maybe,would not if I was not 100% sure that the disc...,"Yes, I think it would",Maybe,Neutral,No,NaN,No,Yes,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",NaN,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",1,Sometimes,Sometimes,43,male,United Kingdom,NaN,United Kingdom,NaN,Supervisor/Team Lead,Sometimes
4,0,6-25,0.0,1.0,Yes,Yes,No,No,No,Neither easy nor difficult,Yes,Maybe,Maybe,No,No,No,NaN,NaN,NaN,NaN,Na

### Downsizing the Dataframe

First, we're going to drop anyone who is self-employed as we are only interested in assessing mental health in tech companies. Next, we're going to drop some questions that we don't believe add substantial information. We've decided to drop "Country Live" and "State Live" since we only care about where each respondent works, and we will also drop the "why not" columns as those cannot be encoded. We will also drop "MH Interference Treatment" and "MH Interference No Treatment" because our group is interested in whether or not mental health is more prevalent in certain demographics, and less concerned about treatment. Finally, since there is a lot of missing data, we will drop any column that is more than 50% empty.

In [ ]:
df.drop(df.loc[df["Self Employed"]==1].index, inplace=True)

In [ ]:
df

,Self Employed,Total Employees,Tech/No Tech,Tech Role,MH Benefits,Awareness of Company MH Care,Formal MH Discussion,MH Resources,Anonymity,Medical Leave,MH Negative Impact,PH Negative Impact,Telling Coworkers,Telling Supervisors,MH PH Equal,Observed Negative Impact,Medical Coverage,Awareness of Local Resources,Reveal Clients (If Diagnosed),Negative Impact (If Revealed Clients),Reveal Employees (If Diagnosed),Negative Impact (If Revealed Employees),Affected Productivity,% Affected,Previous Employers,Prev MH Benefits,Prev Awareness of Company MH Care,Prev Formal MH Discussion,Prev MH Resources,Prev Anonymity,Prev MH Negative Impact,Prev PH Negative Impact,Prev Telling Coworkers,Prev Telling Supervisors,Prev MH PH Equal,Prev Observed Negative Impact,PH Potential Employer,PH Why/Not,MH Potential Employer,MH Why/Not,MH Identification Career Harm,Negative Impact Coworkers,Sharing w/ Family/Friends,Prev Observed Unsupportive Response,Less Likely to Reveal,MH Family History,Past Mental Disorder,Current Mental Disorder,Condition,Maybe Condition,Professional Diagnosis,Diagnosis,Sought Treatment,MH Interference Treatment,MH Interference No Treatment,Age,Gender,Country Live,State Live,Country,State,Position,Remote
0,0,26-100,1.0,NaN,Not eligible for coverage / N/A,NaN,No,No,I don't know,Very easy,No,No,Maybe,Yes,I don't know,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,"No, none did",N/A (not currently aware),I don't know,None did,I don't know,Some of them,None of them,Some of my previous employers,Some of my previous employers,I don't know,None of them,Maybe,NaN,Maybe,NaN,Maybe,"No, I don't think they would",Somewhat open,No,NaN,No,Yes,No,NaN,NaN,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",0,Not applicable to me,Not applicable to me,39,Male,United Kingdom,NaN,United Kingdom,NaN,Back-end Developer,Sometimes
1,0,6-25,1.0,NaN,No,Yes,Yes,Yes,Yes,Somewhat easy,No,No,Maybe,Yes,Yes,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,"Yes, they all did",I was aware of some,None did,Some did,"Yes, always",None of them,None of them,"No, at none of my previous employers",Some of my previous employers,Some did,None of them,Maybe,It would depend on the health issue. If there ...,No,While mental health has become a more prominen...,"No, I don't think it would","No, I don't think they would",Somewhat open,No,NaN,Yes,Yes,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",NaN,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",1,Rarely,Sometimes,29,male,United States of America,Illinois,United States of America,Illinois,Back-end Developer|Front-end Developer,Never
2,0,6-25,1.0,NaN,No,NaN,No,No,I don't know,Neither easy nor difficult,Maybe,No,Maybe,Maybe,I don't know,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,"No, none did",N/A (not currently aware),None did,Some did,I don't know,I don't know,Some of them,Some of my previous employers,I don't know,I don't know,Some of them,Yes,"They would provable need to know, to Judge if ...",Yes,"Stigma, mainly.",Maybe,Maybe,Somewhat open,Maybe/Not sure,Yes,No,Maybe,No,NaN,NaN,No,NaN,1,Not applicable to me,Not applicable to me,38,Male,United Kingdom,NaN,United Kingdom,NaN,Back-end Developer,Always
4,0,6-25,0.0,1.0,Yes,Yes,No,No,No,Neither easy nor difficult,Yes,Maybe,Maybe,No,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,I don't know,N/A (not currently aware),Some did,None did,I don't know,Some of them,Some of them,"No, at none of my previous employers",Some of my previous employers,Some did,Some of them,Maybe,Depending on the interview stage and whether I...,No,I don't know,"Yes, I think it would",Maybe,Somewhat open,"Yes, I experienced",Yes,Yes,Yes,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",NaN,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",1,Sometimes,Sometimes,43,Female,United States of America,Illinois,United States of America,Illinois,Executive Leadership|Supervisor/Team Lead|Dev ...,Sometimes
5,0,More than 1000,1.0,NaN,Yes,I am not sure,No,Yes,Yes,Somewhat easy,Yes,Yes,Maybe,Yes,No,Yes,NaN,NaN,NaN,NaN,Na

In [ ]:
half_empty_columns = []
#go through every column 
for i in list(df):
    if sum(pd.isnull(df[i])) > (len(df)/2):
        half_empty_columns.append(i)
        
half_empty_columns

['Tech Role',
 'Medical Coverage',
 'Awareness of Local Resources',
 'Reveal Clients (If Diagnosed)',
 'Negative Impact (If Revealed Clients)',
 'Reveal Employees (If Diagnosed)',
 'Negative Impact (If Revealed Employees)',
 'Affected Productivity',
 '% Affected',
 'Less Likely to Reveal',
 'Condition',
 'Maybe Condition',
 'Diagnosis']

In [ ]:
df.drop(['PH Why/Not', 'MH Why/Not', 'Country Live', 'State Live', 'MH Interference Treatment', 'MH Interference No Treatment', 'Tech Role',
              'Medical Coverage', 'Awareness of Local Resources', 'Reveal Clients (If Diagnosed)', 'Negative Impact (If Revealed Clients)',
              'Reveal Employees (If Diagnosed)', 'Negative Impact (If Revealed Employees)', 'Affected Productivity',
              '% Affected', 'Maybe Condition', 'Position'], axis=1, inplace=True)

In [ ]:
df.head()

,Self Employed,Total Employees,Tech/No Tech,MH Benefits,Awareness of Company MH Care,Formal MH Discussion,MH Resources,Anonymity,Medical Leave,MH Negative Impact,PH Negative Impact,Telling Coworkers,Telling Supervisors,MH PH Equal,Observed Negative Impact,Previous Employers,Prev MH Benefits,Prev Awareness of Company MH Care,Prev Formal MH Discussion,Prev MH Resources,Prev Anonymity,Prev MH Negative Impact,Prev PH Negative Impact,Prev Telling Coworkers,Prev Telling Supervisors,Prev MH PH Equal,Prev Observed Negative Impact,PH Potential Employer,MH Potential Employer,MH Identification Career Harm,Negative Impact Coworkers,Sharing w/ Family/Friends,Prev Observed Unsupportive Response,Less Likely to Reveal,MH Family History,Past Mental Disorder,Current Mental Disorder,Condition,Professional Diagnosis,Diagnosis,Sought Treatment,Age,Gender,Country,State,Remote
0,0,26-100,1.0,Not eligible for coverage / N/A,NaN,No,No,I don't know,Very easy,No,No,Maybe,Yes,I don't know,No,1,"No, none did",N/A (not currently aware),I don't know,None did,I don't know,Some of them,None of them,Some of my previous employers,Some of my previous employers,I don't know,None of them,Maybe,Maybe,Maybe,"No, I don't think they would",Somewhat open,No,NaN,No,Yes,No,NaN,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",0,39,Male,United Kingdom,NaN,Sometimes
1,0,6-25,1.0,No,Yes,Yes,Yes,Yes,Somewhat easy,No,No,Maybe,Yes,Yes,No,1,"Yes, they all did",I was aware of some,None did,Some did,"Yes, always",None of them,None of them,"No, at none of my previous employers",Some of my previous employers,Some did,None of them,Maybe,No,"No, I don't think it would","No, I don't think they would",Somewhat open,No,NaN,Yes,Yes,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",1,29,male,United States of America,Illinois,Never
2,0,6-25,1.0,No,NaN,No,No,I don't know,Neither easy nor difficult,Maybe,No,Maybe,Maybe,I don't know,No,1,"No, none did",N/A (not currently aware),None did,Some did,I don't know,I don't know,Some of them,Some of my previous employers,I don't know,I don't know,Some of them,Yes,Yes,Maybe,Maybe,Somewhat open,Maybe/Not sure,Yes,No,Maybe,No,NaN,No,NaN,1,38,Male,United Kingdom,NaN,Always
4,0,6-25,0.0,Yes,Yes,No,No,No,Neither easy nor difficult,Yes,Maybe,Maybe,No,No,No,1,I don't know,N/A (not currently aware),Some did,None did,I don't know,Some of them,Some of them,"No, at none of my previous employers",Some of my previous employers,Some did,Some of them,Maybe,No,"Yes, I think it would",Maybe,Somewhat open,"Yes, I experienced",Yes,Yes,Yes,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",1,43,Female,United States of America,Illinois,Sometimes
5,0,More than 1000,1.0,Yes,I am not sure,No,Yes,Yes,Somewhat easy,Yes,Yes,Maybe,Yes,No,Yes,1,"No, none did","Yes, I was aware of all of them",None did,None did,I don't know,"Yes, all of them",Some of them,"No, at none of my previous employers","No, at none of my previous employers",None did,Some of them,Yes,Maybe,"Yes, I think it would",Maybe,Somewhat open,"Yes, I experienced",No,No,No,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",No,NaN,1,42,Male,United Kingdom,NaN,Sometimes


The gender column was a free response field - thus, there were many unique values to parse through.

In [ ]:
df["Gender"].unique()

array(['Male', 'male', 'Male ', 'Female', 'M', 'female', 'm',
       'I identify as female.', 'Bigender', 'non-binary',
       'Female assigned at birth ', 'F', 'Woman', 'man', 'fm', 'f',
       'Cis female ', 'Transitioned, M2F', 'Genderfluid (born female)',
       'Other/Transfeminine', 'Female or Multi-Gender Femme', 'woman',
       'female/woman', 'Cis male', 'Male.', 'Androgynous',
       'male 9:1 female, roughly', nan, 'Male (cis)', 'Other', 'Female ',
       'nb masculine', 'Cisgender Female', 'Man', 'Sex is male',
       'none of your business', 'genderqueer', 'cis male', 'Human',
       'Genderfluid', 'Enby', 'genderqueer woman', 'female ', 'Queer',
       'Agender', 'Dude', 'mail', 'Male/genderqueer', 'fem', 'Nonbinary',
       'male ', 'Female (props for making this a freeform field, though)',
       ' Female', 'Cis Male', 'Male (trans, FtM)', 'Cis-woman',
       'Genderqueer', 'cisdude', 'Genderflux demi-girl',
       'female-bodied; no feelings about gender', 'cis man',
 

In [ ]:
male = ["Male", "male", "Male ", "M", "m", "man", "Cis male", "Male.",
        "Male (cis)", "Man", "Sex is male", "cis male", "Malr", "Dude",
        "I'm a man why didn't you make this a drop down question. You should of asked sex? And I would of answered yes please. Seriously how much text can this take? ",
        "mail", "M|", "male ", "Cis Male", "cisdude", "cis man", "MALE", "Male (trans, FtM)", "Male/genderqueer"]

female = ["Female", "female", "I identify as female.", "female ",
         "Female assigned at birth ", "F", "Woman", "fm", "f", "Cis female",
         "Transitioned, M2F", "Female or Multi-Gender Femme", "Female ",
         "woman", "female/woman", "Cisgender Female", "mtf", "fem",
         "Female (props for making this a freeform field, though)",
         " Female", "Cis-woman", "AFAB", "Transgender woman", "Cis female "]

other = ["Bigender", "non-binary", "Genderfluid (born female)",
        "Other/Transfeminine", "Androgynous", "male 9:1 female, roughly",
        "nb masculine", "genderqueer", "Human", "Genderfluid", "Enby",
        "genderqueer woman", "Queer", "Agender", "Fluid", "Genderflux demi-girl",
        "female-bodied; no feelings about gender", "non-binary", "Male/genderqueer",
        "Nonbinary", "Other", "none of your business", "Unicorn", "human", "Genderqueer"]

#replacing gender values with 1, 2, and 3
df["Gender"] = df["Gender"].replace(male, 1).replace(female, 2).replace(other, 3)
df.head()


,Self Employed,Total Employees,Tech/No Tech,MH Benefits,Awareness of Company MH Care,Formal MH Discussion,MH Resources,Anonymity,Medical Leave,MH Negative Impact,PH Negative Impact,Telling Coworkers,Telling Supervisors,MH PH Equal,Observed Negative Impact,Previous Employers,Prev MH Benefits,Prev Awareness of Company MH Care,Prev Formal MH Discussion,Prev MH Resources,Prev Anonymity,Prev MH Negative Impact,Prev PH Negative Impact,Prev Telling Coworkers,Prev Telling Supervisors,Prev MH PH Equal,Prev Observed Negative Impact,PH Potential Employer,MH Potential Employer,MH Identification Career Harm,Negative Impact Coworkers,Sharing w/ Family/Friends,Prev Observed Unsupportive Response,Less Likely to Reveal,MH Family History,Past Mental Disorder,Current Mental Disorder,Condition,Professional Diagnosis,Diagnosis,Sought Treatment,Age,Gender,Country,State,Remote
0,0,26-100,1.0,Not eligible for coverage / N/A,NaN,No,No,I don't know,Very easy,No,No,Maybe,Yes,I don't know,No,1,"No, none did",N/A (not currently aware),I don't know,None did,I don't know,Some of them,None of them,Some of my previous employers,Some of my previous employers,I don't know,None of them,Maybe,Maybe,Maybe,"No, I don't think they would",Somewhat open,No,NaN,No,Yes,No,NaN,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",0,39,1.0,United Kingdom,NaN,Sometimes
1,0,6-25,1.0,No,Yes,Yes,Yes,Yes,Somewhat easy,No,No,Maybe,Yes,Yes,No,1,"Yes, they all did",I was aware of some,None did,Some did,"Yes, always",None of them,None of them,"No, at none of my previous employers",Some of my previous employers,Some did,None of them,Maybe,No,"No, I don't think it would","No, I don't think they would",Somewhat open,No,NaN,Yes,Yes,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",1,29,1.0,United States of America,Illinois,Never
2,0,6-25,1.0,No,NaN,No,No,I don't know,Neither easy nor difficult,Maybe,No,Maybe,Maybe,I don't know,No,1,"No, none did",N/A (not currently aware),None did,Some did,I don't know,I don't know,Some of them,Some of my previous employers,I don't know,I don't know,Some of them,Yes,Yes,Maybe,Maybe,Somewhat open,Maybe/Not sure,Yes,No,Maybe,No,NaN,No,NaN,1,38,1.0,United Kingdom,NaN,Always
4,0,6-25,0.0,Yes,Yes,No,No,No,Neither easy nor difficult,Yes,Maybe,Maybe,No,No,No,1,I don't know,N/A (not currently aware),Some did,None did,I don't know,Some of them,Some of them,"No, at none of my previous employers",Some of my previous employers,Some did,Some of them,Maybe,No,"Yes, I think it would",Maybe,Somewhat open,"Yes, I experienced",Yes,Yes,Yes,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",1,43,2.0,United States of America,Illinois,Sometimes
5,0,More than 1000,1.0,Yes,I am not sure,No,Yes,Yes,Somewhat easy,Yes,Yes,Maybe,Yes,No,Yes,1,"No, none did","Yes, I was aware of all of them",None did,None did,I don't know,"Yes, all of them",Some of them,"No, at none of my previous employers","No, at none of my previous employers",None did,Some of them,Yes,Maybe,"Yes, I think it would",Maybe,Somewhat open,"Yes, I experienced",No,No,No,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",No,NaN,1,42,1.0,United Kingdom,NaN,Sometimes


In [ ]:
len(df)

1146

In [ ]:
#fill NaN values with 3
df = df[~df["Gender"].isna()]
df

,Self Employed,Total Employees,Tech/No Tech,MH Benefits,Awareness of Company MH Care,Formal MH Discussion,MH Resources,Anonymity,Medical Leave,MH Negative Impact,PH Negative Impact,Telling Coworkers,Telling Supervisors,MH PH Equal,Observed Negative Impact,Previous Employers,Prev MH Benefits,Prev Awareness of Company MH Care,Prev Formal MH Discussion,Prev MH Resources,Prev Anonymity,Prev MH Negative Impact,Prev PH Negative Impact,Prev Telling Coworkers,Prev Telling Supervisors,Prev MH PH Equal,Prev Observed Negative Impact,PH Potential Employer,MH Potential Employer,MH Identification Career Harm,Negative Impact Coworkers,Sharing w/ Family/Friends,Prev Observed Unsupportive Response,Less Likely to Reveal,MH Family History,Past Mental Disorder,Current Mental Disorder,Condition,Professional Diagnosis,Diagnosis,Sought Treatment,Age,Gender,Country,State,Remote
0,0,26-100,1.0,Not eligible for coverage / N/A,NaN,No,No,I don't know,Very easy,No,No,Maybe,Yes,I don't know,No,1,"No, none did",N/A (not currently aware),I don't know,None did,I don't know,Some of them,None of them,Some of my previous employers,Some of my previous employers,I don't know,None of them,Maybe,Maybe,Maybe,"No, I don't think they would",Somewhat open,No,NaN,No,Yes,No,NaN,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",0,39,1.0,United Kingdom,NaN,Sometimes
1,0,6-25,1.0,No,Yes,Yes,Yes,Yes,Somewhat easy,No,No,Maybe,Yes,Yes,No,1,"Yes, they all did",I was aware of some,None did,Some did,"Yes, always",None of them,None of them,"No, at none of my previous employers",Some of my previous employers,Some did,None of them,Maybe,No,"No, I don't think it would","No, I don't think they would",Somewhat open,No,NaN,Yes,Yes,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",1,29,1.0,United States of America,Illinois,Never
2,0,6-25,1.0,No,NaN,No,No,I don't know,Neither easy nor difficult,Maybe,No,Maybe,Maybe,I don't know,No,1,"No, none did",N/A (not currently aware),None did,Some did,I don't know,I don't know,Some of them,Some of my previous employers,I don't know,I don't know,Some of them,Yes,Yes,Maybe,Maybe,Somewhat open,Maybe/Not sure,Yes,No,Maybe,No,NaN,No,NaN,1,38,1.0,United Kingdom,NaN,Always
4,0,6-25,0.0,Yes,Yes,No,No,No,Neither easy nor difficult,Yes,Maybe,Maybe,No,No,No,1,I don't know,N/A (not currently aware),Some did,None did,I don't know,Some of them,Some of them,"No, at none of my previous employers",Some of my previous employers,Some did,Some of them,Maybe,No,"Yes, I think it would",Maybe,Somewhat open,"Yes, I experienced",Yes,Yes,Yes,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",1,43,2.0,United States of America,Illinois,Sometimes
5,0,More than 1000,1.0,Yes,I am not sure,No,Yes,Yes,Somewhat easy,Yes,Yes,Maybe,Yes,No,Yes,1,"No, none did","Yes, I was aware of all of them",None did,None did,I don't know,"Yes, all of them",Some of them,"No, at none of my previous employers","No, at none of my previous employers",None did,Some of them,Yes,Maybe,"Yes, I think it would",Maybe,Somewhat open,"Yes, I experienced",No,No,No,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",No,NaN,1,42,1.0,United Kingdom,NaN,Sometimes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1426,0,100-500,1.0,I don't know,I am not sure,No,I don't know,I don't know,Somewhat easy,Maybe,No,Maybe,Maybe,I don't know,No,1,I don't know,N/A (not currently aware),I don't know,Some did,I don't know,Some of them,Some of them,Some of my previous employers,Some of my previous employers,Some did,None of them,Maybe,Maybe,"Yes, it has",Maybe,Somewhat open,"Yes, I experienced",Maybe,I don't know,Yes,Yes,Attention Deficit Hyperactivity Disorder,Yes,Attention Deficit Hyperactivity Disorder,1,26,2.0,Canada,NaN,Sometimes
1427,0,500-1000,1.0,Yes,No,No,N

Taking a look at the Age column, there are some NaN values and others that might have been mistakes (for example, no 3 year old should be working full-time, and there exists no 323 year old). We will fill outlying ages with the mean, and separate this column into different age groups. 

In [ ]:
df["Age"].isnull().sum()

0

In [ ]:
print(min(df["Age"]))
print(max(df["Age"]))

3
323


In [ ]:
mean_age = df.mean()["Age"]
mean_age

33.67804024496938

In [ ]:
df.loc[df["Age"] < 13, "Age"] = mean_age #filling wrong values with mean age
df.loc[df["Age"] > 72, "Age"] = mean_age #filling wrong values with mean age

/shared-libs/python3.7/py/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/shared-libs/python3.7/py/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [ ]:
# splitting Age feature up into different age groups to refine the feature
# separated them until the groups were split more evenly 

df["Age"] = df["Age"].astype(int) 

df.loc[(df["Age"] >= 13) & (df["Age"] <= 21), "Age"] = 1 #adolescents
df.loc[(df["Age"] > 21) & (df["Age"] <= 25), "Age"] = 2 #younger adults
df.loc[(df["Age"] > 25) & (df["Age"] <= 35), "Age"] = 3 #adults
df.loc[(df["Age"] > 35) & (df["Age"] <= 45), "Age"] = 4 #middle-aged adults
df.loc[df["Age"] > 45, "Age"] = 5 #older adults

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/shared-libs/python3.7/py/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/shared-libs/python3.7/py/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

In [ ]:
df.head()

,Self Employed,Total Employees,Tech/No Tech,MH Benefits,Awareness of Company MH Care,Formal MH Discussion,MH Resources,Anonymity,Medical Leave,MH Negative Impact,PH Negative Impact,Telling Coworkers,Telling Supervisors,MH PH Equal,Observed Negative Impact,Previous Employers,Prev MH Benefits,Prev Awareness of Company MH Care,Prev Formal MH Discussion,Prev MH Resources,Prev Anonymity,Prev MH Negative Impact,Prev PH Negative Impact,Prev Telling Coworkers,Prev Telling Supervisors,Prev MH PH Equal,Prev Observed Negative Impact,PH Potential Employer,MH Potential Employer,MH Identification Career Harm,Negative Impact Coworkers,Sharing w/ Family/Friends,Prev Observed Unsupportive Response,Less Likely to Reveal,MH Family History,Past Mental Disorder,Current Mental Disorder,Condition,Professional Diagnosis,Diagnosis,Sought Treatment,Age,Gender,Country,State,Remote
0,0,26-100,1.0,Not eligible for coverage / N/A,NaN,No,No,I don't know,Very easy,No,No,Maybe,Yes,I don't know,No,1,"No, none did",N/A (not currently aware),I don't know,None did,I don't know,Some of them,None of them,Some of my previous employers,Some of my previous employers,I don't know,None of them,Maybe,Maybe,Maybe,"No, I don't think they would",Somewhat open,No,NaN,No,Yes,No,NaN,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",0,4,1.0,United Kingdom,NaN,Sometimes
1,0,6-25,1.0,No,Yes,Yes,Yes,Yes,Somewhat easy,No,No,Maybe,Yes,Yes,No,1,"Yes, they all did",I was aware of some,None did,Some did,"Yes, always",None of them,None of them,"No, at none of my previous employers",Some of my previous employers,Some did,None of them,Maybe,No,"No, I don't think it would","No, I don't think they would",Somewhat open,No,NaN,Yes,Yes,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",1,3,1.0,United States of America,Illinois,Never
2,0,6-25,1.0,No,NaN,No,No,I don't know,Neither easy nor difficult,Maybe,No,Maybe,Maybe,I don't know,No,1,"No, none did",N/A (not currently aware),None did,Some did,I don't know,I don't know,Some of them,Some of my previous employers,I don't know,I don't know,Some of them,Yes,Yes,Maybe,Maybe,Somewhat open,Maybe/Not sure,Yes,No,Maybe,No,NaN,No,NaN,1,4,1.0,United Kingdom,NaN,Always
4,0,6-25,0.0,Yes,Yes,No,No,No,Neither easy nor difficult,Yes,Maybe,Maybe,No,No,No,1,I don't know,N/A (not currently aware),Some did,None did,I don't know,Some of them,Some of them,"No, at none of my previous employers",Some of my previous employers,Some did,Some of them,Maybe,No,"Yes, I think it would",Maybe,Somewhat open,"Yes, I experienced",Yes,Yes,Yes,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",1,4,2.0,United States of America,Illinois,Sometimes
5,0,More than 1000,1.0,Yes,I am not sure,No,Yes,Yes,Somewhat easy,Yes,Yes,Maybe,Yes,No,Yes,1,"No, none did","Yes, I was aware of all of them",None did,None did,I don't know,"Yes, all of them",Some of them,"No, at none of my previous employers","No, at none of my previous employers",None did,Some of them,Yes,Maybe,"Yes, I think it would",Maybe,Somewhat open,"Yes, I experienced",No,No,No,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",No,NaN,1,4,1.0,United Kingdom,NaN,Sometimes


In [ ]:
df["Age"].value_counts()

3    615
4    305
2    125
5     77
1     21
Name: Age, dtype: int64

Here, we split up the "Total Employees" column into the 6 groupings available on the dataset.

In [ ]:
#split Total Employees feature into 6 groups based on the size of each respondent's company

df.loc[(df["Total Employees"] == "1-5"), "Total Employees"] = 1 #extremely small
df.loc[(df["Total Employees"] == "6-25"), "Total Employees"] = 2 #very small companies
df.loc[(df["Total Employees"] == "26-100"), "Total Employees"] = 3 #small companies
df.loc[(df["Total Employees"] == "100-500"), "Total Employees"] = 4 #medium companies
df.loc[(df["Total Employees"] == "500-1000"), "Total Employees"] = 5 #large companies
df.loc[(df["Total Employees"] == "More than 1000"), "Total Employees"] = 6 #very large companies

/shared-libs/python3.7/py/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [ ]:
mode_size = df["Total Employees"].mode()[0]
df["Total Employees"] = df["Total Employees"].fillna(mode_size)

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df["Country"].value_counts()

United States of America    714
United Kingdom              132
Canada                       61
Germany                      44
Netherlands                  31
Australia                    26
Sweden                       15
France                       11
Ireland                      10
Brazil                        9
Russia                        9
India                         7
New Zealand                   6
Bulgaria                      6
Switzerland                   6
Finland                       5
Denmark                       5
South Africa                  4
Belgium                       3
Norway                        3
Italy                         3
Chile                         3
Spain                         2
Estonia                       2
Mexico                        2
Pakistan                      2
Colombia                      2
Israel                        2
Poland                        2
Bosnia and Herzegovina        2
Romania                       1
Japan   

Most countries only have <=20 respondents, so those respondents cannot accurately represent their respective countries. We will group these countries into 1 group, and keep the other countries (with more than 30 respondents) as their own groupings.

In [ ]:
def country_encoding(country):
    if country == "United States of America":
        return 1
    elif country == "United Kingdom":
        return 2
    elif country == "Canada":
        return 3
    elif country == "Germany":
        return 4
    elif country == "Netherlands":
        return 5
    elif country == "Australia":
        return 6
    else:
        return 7

In [ ]:
df["Country"] = df["Country"].apply(country_encoding)

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df.head()

,Self Employed,Total Employees,Tech/No Tech,MH Benefits,Awareness of Company MH Care,Formal MH Discussion,MH Resources,Anonymity,Medical Leave,MH Negative Impact,PH Negative Impact,Telling Coworkers,Telling Supervisors,MH PH Equal,Observed Negative Impact,Previous Employers,Prev MH Benefits,Prev Awareness of Company MH Care,Prev Formal MH Discussion,Prev MH Resources,Prev Anonymity,Prev MH Negative Impact,Prev PH Negative Impact,Prev Telling Coworkers,Prev Telling Supervisors,Prev MH PH Equal,Prev Observed Negative Impact,PH Potential Employer,MH Potential Employer,MH Identification Career Harm,Negative Impact Coworkers,Sharing w/ Family/Friends,Prev Observed Unsupportive Response,Less Likely to Reveal,MH Family History,Past Mental Disorder,Current Mental Disorder,Condition,Professional Diagnosis,Diagnosis,Sought Treatment,Age,Gender,Country,State,Remote
0,0,3,1.0,Not eligible for coverage / N/A,NaN,No,No,I don't know,Very easy,No,No,Maybe,Yes,I don't know,No,1,"No, none did",N/A (not currently aware),I don't know,None did,I don't know,Some of them,None of them,Some of my previous employers,Some of my previous employers,I don't know,None of them,Maybe,Maybe,Maybe,"No, I don't think they would",Somewhat open,No,NaN,No,Yes,No,NaN,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",0,4,1.0,2,NaN,Sometimes
1,0,2,1.0,No,Yes,Yes,Yes,Yes,Somewhat easy,No,No,Maybe,Yes,Yes,No,1,"Yes, they all did",I was aware of some,None did,Some did,"Yes, always",None of them,None of them,"No, at none of my previous employers",Some of my previous employers,Some did,None of them,Maybe,No,"No, I don't think it would","No, I don't think they would",Somewhat open,No,NaN,Yes,Yes,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",1,3,1.0,1,Illinois,Never
2,0,2,1.0,No,NaN,No,No,I don't know,Neither easy nor difficult,Maybe,No,Maybe,Maybe,I don't know,No,1,"No, none did",N/A (not currently aware),None did,Some did,I don't know,I don't know,Some of them,Some of my previous employers,I don't know,I don't know,Some of them,Yes,Yes,Maybe,Maybe,Somewhat open,Maybe/Not sure,Yes,No,Maybe,No,NaN,No,NaN,1,4,1.0,2,NaN,Always
4,0,2,0.0,Yes,Yes,No,No,No,Neither easy nor difficult,Yes,Maybe,Maybe,No,No,No,1,I don't know,N/A (not currently aware),Some did,None did,I don't know,Some of them,Some of them,"No, at none of my previous employers",Some of my previous employers,Some did,Some of them,Maybe,No,"Yes, I think it would",Maybe,Somewhat open,"Yes, I experienced",Yes,Yes,Yes,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",1,4,2.0,1,Illinois,Sometimes
5,0,6,1.0,Yes,I am not sure,No,Yes,Yes,Somewhat easy,Yes,Yes,Maybe,Yes,No,Yes,1,"No, none did","Yes, I was aware of all of them",None did,None did,I don't know,"Yes, all of them",Some of them,"No, at none of my previous employers","No, at none of my previous employers",None did,Some of them,Yes,Maybe,"Yes, I think it would",Maybe,Somewhat open,"Yes, I experienced",No,No,No,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",No,NaN,1,4,1.0,2,NaN,Sometimes


In [ ]:
df["Country"].value_counts()

1    714
7    135
2    132
3     61
4     44
5     31
6     26
Name: Country, dtype: int64

In [ ]:
df["State"].isnull().sum()  

429

In [ ]:
df["State"] = df["State"].fillna(0) #fill NaN values with 0 to group respondents outside the US 

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Here, we're going to split up the states into 4 main regions (Northeast, Midwest, South, West) as designated by the United States Census Bureau.

In [ ]:
def state_encoding(state):
    northeast = ["Maine", "New Hampshire", "Vermont", "Massachusetts", "Rhode Island",
                "Connecticut", "New York", "Pennsylvania", "New Jersey"]
    midwest = ["Wisconsin", "Michigan", "Illinois", "Indiana", "Ohio", "North Dakota",
                "South Dakota", "Nebraska", "Kansas", "Minnesota", "Iowa", "Missouri"]
    south = ["Delaware", "Maryland", "District of Columbia", "Virginia", "West Virginia",
            "North Carolina", "South Carolina", "Georgia", "Florida", "Kentucky", "Tennessee",
            "Mississippi", "Alabama", "Oklahoma", "Texas", "Arkansas", "Louisiana"]
    west = ["Idaho", "Montana", "Wyoming", "Nevada", "Utah", "Colorado", "Arizona",
            "New Mexico", "Alaska", "Washington", "Oregon", "California", "Hawaii"]

    if state in northeast:
        return 1
    elif state in midwest:
        return 2
    elif state in south:
        return 3
    elif state in west:
        return 4
    else:
        return 0 #NaN values / respondents outside the United States

In [ ]:
df["State"] = df["State"].apply(state_encoding)

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df["State"].value_counts()

0    429
2    221
4    218
3    167
1    108
Name: State, dtype: int64

In [ ]:
df

,Self Employed,Total Employees,Tech/No Tech,MH Benefits,Awareness of Company MH Care,Formal MH Discussion,MH Resources,Anonymity,Medical Leave,MH Negative Impact,PH Negative Impact,Telling Coworkers,Telling Supervisors,MH PH Equal,Observed Negative Impact,Previous Employers,Prev MH Benefits,Prev Awareness of Company MH Care,Prev Formal MH Discussion,Prev MH Resources,Prev Anonymity,Prev MH Negative Impact,Prev PH Negative Impact,Prev Telling Coworkers,Prev Telling Supervisors,Prev MH PH Equal,Prev Observed Negative Impact,PH Potential Employer,MH Potential Employer,MH Identification Career Harm,Negative Impact Coworkers,Sharing w/ Family/Friends,Prev Observed Unsupportive Response,Less Likely to Reveal,MH Family History,Past Mental Disorder,Current Mental Disorder,Condition,Professional Diagnosis,Diagnosis,Sought Treatment,Age,Gender,Country,State,Remote
0,0,3,1.0,Not eligible for coverage / N/A,NaN,No,No,I don't know,Very easy,No,No,Maybe,Yes,I don't know,No,1,"No, none did",N/A (not currently aware),I don't know,None did,I don't know,Some of them,None of them,Some of my previous employers,Some of my previous employers,I don't know,None of them,Maybe,Maybe,Maybe,"No, I don't think they would",Somewhat open,No,NaN,No,Yes,No,NaN,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",0,4,1.0,2,0,Sometimes
1,0,2,1.0,No,Yes,Yes,Yes,Yes,Somewhat easy,No,No,Maybe,Yes,Yes,No,1,"Yes, they all did",I was aware of some,None did,Some did,"Yes, always",None of them,None of them,"No, at none of my previous employers",Some of my previous employers,Some did,None of them,Maybe,No,"No, I don't think it would","No, I don't think they would",Somewhat open,No,NaN,Yes,Yes,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",1,3,1.0,1,2,Never
2,0,2,1.0,No,NaN,No,No,I don't know,Neither easy nor difficult,Maybe,No,Maybe,Maybe,I don't know,No,1,"No, none did",N/A (not currently aware),None did,Some did,I don't know,I don't know,Some of them,Some of my previous employers,I don't know,I don't know,Some of them,Yes,Yes,Maybe,Maybe,Somewhat open,Maybe/Not sure,Yes,No,Maybe,No,NaN,No,NaN,1,4,1.0,2,0,Always
4,0,2,0.0,Yes,Yes,No,No,No,Neither easy nor difficult,Yes,Maybe,Maybe,No,No,No,1,I don't know,N/A (not currently aware),Some did,None did,I don't know,Some of them,Some of them,"No, at none of my previous employers",Some of my previous employers,Some did,Some of them,Maybe,No,"Yes, I think it would",Maybe,Somewhat open,"Yes, I experienced",Yes,Yes,Yes,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",1,4,2.0,1,2,Sometimes
5,0,6,1.0,Yes,I am not sure,No,Yes,Yes,Somewhat easy,Yes,Yes,Maybe,Yes,No,Yes,1,"No, none did","Yes, I was aware of all of them",None did,None did,I don't know,"Yes, all of them",Some of them,"No, at none of my previous employers","No, at none of my previous employers",None did,Some of them,Yes,Maybe,"Yes, I think it would",Maybe,Somewhat open,"Yes, I experienced",No,No,No,Yes,"Anxiety Disorder (Generalized, Social, Phobia,...",No,NaN,1,4,1.0,2,0,Sometimes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1426,0,4,1.0,I don't know,I am not sure,No,I don't know,I don't know,Somewhat easy,Maybe,No,Maybe,Maybe,I don't know,No,1,I don't know,N/A (not currently aware),I don't know,Some did,I don't know,Some of them,Some of them,Some of my previous employers,Some of my previous employers,Some did,None of them,Maybe,Maybe,"Yes, it has",Maybe,Somewhat open,"Yes, I experienced",Maybe,I don't know,Yes,Yes,Attention Deficit Hyperactivity Disorder,Yes,Attention Deficit Hyperactivity Disorder,1,3,2.0,3,0,Sometimes
1427,0,5,1.0,Yes,No,No,No,Yes,Somewhat easy,No,No,Yes,Yes,Yes,No,1,Some did,I was aware of some,None did,None did,I don't know,Some of them,None of them,Some of my previous employers

### Encoding 

In [ ]:
# Construct data frame that shows number of unique values for each column
d = {'Column Name': df.columns, 'Unique Values': df.nunique()}
unique_vals = pd.DataFrame(data=d).reset_index()[['Column Name', 'Unique Values']].sort_values(by='Unique Values', ascending=False)

# Selecting only column names that have a valid number of unique values for dummy variable transformation (filter out binary and large numbers of unique values)
valid_dummy = unique_vals[(unique_vals['Unique Values']!=2) & (unique_vals['Unique Values']!=53) & (unique_vals['Unique Values']!=48)]
dummy_list = valid_dummy['Column Name'].values
dummy_list

array(['Condition', 'Diagnosis', 'Country', 'Medical Leave',
       'Total Employees', 'Sharing w/ Family/Friends',
       'Negative Impact Coworkers', 'State', 'Age',
       'MH Identification Career Harm',
       'Prev Observed Unsupportive Response', 'Prev Formal MH Discussion',
       'Prev Awareness of Company MH Care', 'Prev MH Benefits',
       'Prev Telling Supervisors', 'Prev MH Negative Impact',
       'Prev Anonymity', 'Prev MH PH Equal', 'MH Benefits',
       'Prev Telling Coworkers', 'Less Likely to Reveal',
       'PH Potential Employer', 'MH Family History',
       'Past Mental Disorder', 'Current Mental Disorder', 'Gender',
       'MH Potential Employer', 'Remote', 'Prev Observed Negative Impact',
       'PH Negative Impact', 'Awareness of Company MH Care',
       'Formal MH Discussion', 'MH Resources', 'Anonymity',
       'Prev PH Negative Impact', 'MH Negative Impact',
       'Telling Coworkers', 'Telling Supervisors', 'MH PH Equal',
       'Prev MH Resources', 'Self 

In [ ]:
df.columns 

l = []
for col in df.columns:
    if col not in dummy_list:
        l.append(col)
l

['Tech/No Tech',
 'Observed Negative Impact',
 'Previous Employers',
 'Professional Diagnosis',
 'Sought Treatment']

In [ ]:
df.to_csv("mental_health_tech_current.csv")

In [ ]:
# Make dummy variable columns for the valid column names we found above
df = pd.get_dummies(df, columns=dummy_list)
df.head()

,Tech/No Tech,Observed Negative Impact,Previous Employers,Professional Diagnosis,Sought Treatment,Condition_Addictive Disorder,"Condition_Anxiety Disorder (Generalized, Social, Phobia, etc)","Condition_Anxiety Disorder (Generalized, Social, Phobia, etc)|Asperges","Condition_Anxiety Disorder (Generalized, Social, Phobia, etc)|Attention Deficit Hyperactivity Disorder","Condition_Anxiety Disorder (Generalized, Social, Phobia, etc)|Attention Deficit Hyperactivity Disorder|Obsessive-Compulsive Disorder","Condition_Anxiety Disorder (Generalized, Social, Phobia, etc)|Attention Deficit Hyperactivity Disorder|Post-traumatic Stress Disorder","Condition_Anxiety Disorder (Generalized, Social, Phobia, etc)|Attention Deficit Hyperactivity Disorder|Post-traumatic Stress Disorder|Addictive Disorder","Condition_Anxiety Disorder (Generalized, Social, Phobia, etc)|Attention Deficit Hyperactivity Disorder|Stress Response Syndromes","Condition_Anxiety Disorder (Generalized, Social, Phobia, etc)|Eating Disorder (Anorexia, Bulimia, etc)|Stress Response Syndromes","Condition_Anxiety Disorder (Generalized, Social, Phobia, etc)|Mood Disorder (Depression, Bipolar Disorder, etc)","Condition_Anxiety Disorder (Generalized, Social, Phobia, etc)|Mood Disorder (Depression, Bipolar Disorder, etc)|Addictive Disorder","Condition_Anxiety Disorder (Generalized, Social, Phobia, etc)|Mood Disorder (Depression, Bipolar Disorder, etc)|Attention Deficit Hyperactivity Disorder","Condition_Anxiety Disorder (Generalized, Social, Phobia, etc)|Mood Disorder (Depression, Bipolar Disorder, etc)|Attention Deficit Hyperactivity Disorder|Dissociative Disorder","Condition_Anxiety Disorder (Generalized, Social, Phobia, etc)|Mood Disorder (Depression, Bipolar Disorder, etc)|Attention Deficit Hyperactivity Disorder|Obsessive-Compulsive Disorder","Condition_Anxiety Disorder (Generalized, Social, Phobia, etc)|Mood Disorder (Depression, Bipolar Disorder, etc)|Attention Deficit Hyperactivity Disorder|Obsessive-Compulsive Disorder|Substance Use Disorder","Condition_Anxiety Disorder (Generalized, Social, Phobia, etc)|Mood Disorder (Depression, Bipolar Disorder, etc)|Attention Deficit Hyperactivity Disorder|Personality Disorder (Borderline, Antisocial, Paranoid, etc)","Condition_Anxiety Disorder (Generalized, Social, Phobia, etc)|Mood Disorder (Depression, Bipolar Disorder, etc)|Attention Deficit Hyperactivity Disorder|Personality Disorder (Borderline, Antisocial, Paranoid, etc)|Obsessive-Compulsive Disorder|Post-traumatic Stress Disorder","Condition_Anxiety Disorder (Generalized, Social, Phobia, etc)|Mood Disorder (Depression, Bipolar Disorder, etc)|Attention Deficit Hyperactivity Disorder|Personality Disorder (Borderline, Antisocial, Paranoid, etc)|Stress Response Syndromes|Addictive Disorder","Condition_Anxiety Disorder (Generalized, Social, Phobia, etc)|Mood Disorder (Depression, Bipolar Disorder, etc)|Attention Deficit Hyperactivity Disorder|Post-traumatic Stress Disorder","Condition_Anxiety Disorder (Generalized, Social, Phobia, etc)|Mood Disorder (Depression, Bipolar Disorder, etc)|Attention Deficit Hyperactivity Disorder|Post-traumatic Stress Disorder|Substance Use Disorder","Condition_Anxiety Disorder (Generalized, Social, Phobia, etc)|Mood Disorder (Depression, Bipolar Disorder, etc)|Attention Deficit Hyperactivity Disorder|Substance Use Disorder","Condition_Anxiety Disorder (Generalized, Social, Phobia, etc)|Mood Disorder (Depression, Bipolar Disorder, etc)|Dissociative Disorder","Condition_Anxiety Disorder (Generalized, Social, Phobia, etc)|Mood Disorder (Depression, Bipolar Disorder, etc)|Eating Disorder (Anorexia, Bulimia, etc)","Condition_Anxiety Disorder (Generalized, Social, Phobia, etc)|Mood Disorder (Depression, Bipolar Disorder, etc)|Eating Disorder (Anorexia, Bulimia, etc)|Obsessive-Compulsive Disorder","Condition_Anxiety Disorder (Generalized, Social, Phobia, etc)|Mood Disorder (Depression, Bipolar Disorder, etc)|Eating Disorder (Anorexia, Bulimia, etc)|Personality Di

In [ ]:
# Making binary columns within the data frame consistent

df.loc[df['Observed Negative Impact'] == 'Yes', 'Observed Negative Impact'] = 1
df.loc[df['Observed Negative Impact'] == 'No', 'Observed Negative Impact'] = 0


df.loc[df['Tech/No Tech'] == 1.0, 'Tech/No Tech'] = 'Yes'
df.loc[df['Tech/No Tech'] == 0.0, 'Tech/No Tech'] = 'No'

df.loc[df['Tech/No Tech'] == 'Yes', 'Tech/No Tech'] = 1
df.loc[df['Tech/No Tech'] == 'No', 'Tech/No Tech'] = 0

In [ ]:
df.to_csv("mental_health_tech_dummies_current.csv")

In [ ]:
# sns.distplot(viz['Age'], kde=True, rug=True);
# plt.title('Distribution of Company Size')

In [ ]:
import sklearn.model_selection 
from sklearn.model_selection import RepeatedKFold
kf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=None) 

for train_index, test_index in kf.split(df):
      #print("Train:", train_index, "Validation:",test_index)
      X_train, X_test = X[train_index], X[test_index] 
      y_train, y_test = y[train_index], y[test_index]

NameError: name 'X' is not defined

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=387b3f34-ad6b-4fed-8016-3e4e46161261' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>